In [3]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.linear_model import Ridge, Lasso, ElasticNet, TweedieRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, Matern
from pygam import LinearGAM
import statsmodels.api as sm
import itertools
from tqdm import tqdm
import time
import joblib


In [2]:
!pip install pygam -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 2.3 MB/s eta 0:00:00


In [4]:
df = pd.read_csv('/content/research2_data.csv')
df[['sleep_duration', 'time2sleep', 'time2wake']] = df[['sleep_duration', 'time2sleep', 'time2wake']].astype(str).applymap(lambda x: int(x.split('.')[0])*60 + int(x.split('.')[1]))

def cycle_transform(x):
    return np.cos(2 * np.pi * x / 1440)

df['time2sleep'] = df['time2sleep'].apply(cycle_transform)
df['Deep_fraction'] = 100 - df['REM_fraction'] - df['Light_fraction']
df['REM_duration'] = df['REM_fraction'] * df['sleep_duration'] / 100
df['Deep_duration'] = df['Deep_fraction'] * df['sleep_duration'] / 100
df['Light_duration'] = df['Light_fraction'] * df['sleep_duration'] / 100

print(f"Пропуски: {df.isnull().sum().sum()}, Дубликаты: {df.duplicated().sum()}")
df = df.drop('index', axis=1)

all_features = ['sleep_duration', 'time2sleep', 'time_warm_up', 'steps', 'kkal', 'time2wake']
target = 'sleep_score'

Пропуски: 0, Дубликаты: 0


In [5]:
def generate_feature_combinations(base_features):
    optional_features = [f for f in base_features if f != 'time2sleep']
    combinations = []
    for r in range(0, len(optional_features) + 1):
        for comb in itertools.combinations(optional_features, r):
            full_comb = ['time2sleep'] + list(comb)
            if len(full_comb) >= 2:
                combinations.append(full_comb)
    return combinations

feature_combinations = generate_feature_combinations(all_features)

In [6]:
all_models_config = {
    'Linear': {
        'Ridge': Ridge(alpha=12.0, random_state=42),
        'Lasso': Lasso(alpha=0.3, random_state=42, max_iter=10000),
        'ElasticNet': ElasticNet(alpha=0.3, l1_ratio=0.5, random_state=42, max_iter=10000)
    },
    'Tree': {
        'RandomForest': RandomForestRegressor(n_estimators=50, max_depth=3, random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingRegressor(n_estimators=30, max_depth=2, learning_rate=0.1, random_state=42)
    },
    'GAM': {
        'GAM_lam0.1': LinearGAM(n_splines=10, lam=0.1),
        'GAM_lam1': LinearGAM(n_splines=10, lam=1),
        'GAM_lam10': LinearGAM(n_splines=10, lam=10)
    },
    'GLM': {
        'GLM_Tweedie': TweedieRegressor(power=1.5, alpha=0.1)
    },
    'GPR': {
        'GPR_RBF': GaussianProcessRegressor(
            kernel=ConstantKernel(1.0) * RBF(length_scale=1.0),
            alpha=1e-3, n_restarts_optimizer=3, random_state=42, normalize_y=True
        ),
        'GPR_Matern1.5': GaussianProcessRegressor(
            kernel=ConstantKernel(1.0) * Matern(length_scale=1.0, nu=1.5),
            alpha=1e-3, n_restarts_optimizer=3, random_state=42, normalize_y=True
        )
    },
    'Other': {
        'SVR': SVR(kernel='linear', C=1.0),
        'KNN': KNeighborsRegressor(n_neighbors=3),
        'MLP': MLPRegressor(hidden_layer_sizes=(20, 10), max_iter=1000, random_state=42, early_stopping=True)
    }
}

In [7]:
def evaluate_model(model, model_name, model_type, X_train_scaled, X_test_scaled, y_train, y_test):
        if model_type == 'GAM':
            kf = KFold(n_splits=3, shuffle=True, random_state=42)
            cv_scores = []
            for train_idx, val_idx in kf.split(X_train_scaled):
                X_tr, X_val = X_train_scaled[train_idx], X_train_scaled[val_idx]
                y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
                gam_temp = LinearGAM(n_splines=10, lam=model.lam)
                gam_temp.fit(X_tr, y_tr)
                y_val_pred = gam_temp.predict(X_val)
                cv_scores.append(r2_score(y_val, y_val_pred))
            cv_r2 = np.mean(cv_scores)
            cv_std = np.std(cv_scores)

            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)
        else:
            cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='r2')
            cv_r2 = cv_scores.mean()
            cv_std = cv_scores.std()

            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)

        test_r2 = r2_score(y_test, y_pred)
        test_mae = mean_absolute_error(y_test, y_pred)

        return {
            'cv_r2': cv_r2,
            'cv_std': cv_std,
            'test_r2': test_r2,
            'test_mae': test_mae,
            'y_pred': y_pred,
            'model': model
        }


In [8]:
all_results = {}
best_models_by_type = {}
best_overall = None
start_time = time.time()

for comb_id, features in enumerate(tqdm(feature_combinations, desc="Перебор комбинаций")):
    X = df[features]
    y = df[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    for model_type, models_dict in all_models_config.items():
        for model_name, model in models_dict.items():
            full_model_name = f"{model_type}_{model_name}_comb{comb_id}"

            if model_type == 'GAM':
                model_copy = LinearGAM(n_splines=10, lam=model.lam)
            elif model_type == 'GPR':
                if model_name == 'GPR_RBF':
                    model_copy = GaussianProcessRegressor(
                        kernel=ConstantKernel(1.0) * RBF(length_scale=1.0),
                        alpha=1e-3, n_restarts_optimizer=3, random_state=42, normalize_y=True
                    )
                else:
                    model_copy = GaussianProcessRegressor(
                        kernel=ConstantKernel(1.0) * Matern(length_scale=1.0, nu=1.5),
                        alpha=1e-3, n_restarts_optimizer=3, random_state=42, normalize_y=True
                    )
            else:
                model_copy = model.__class__(**model.get_params())

            result_dict = evaluate_model(
                model_copy, model_name, model_type,
                X_train_scaled, X_test_scaled, y_train, y_test
            )

            if result_dict:
                result = {
                    'combination_id': comb_id,
                    'features': features,
                    'num_features': len(features),
                    'cv_r2': result_dict['cv_r2'],
                    'cv_std': result_dict['cv_std'],
                    'test_r2': result_dict['test_r2'],
                    'test_mae': result_dict['test_mae'],
                    'model_type': model_type,
                    'model_name': model_name,
                    'model': result_dict['model']
                }

                all_results[full_model_name] = result

                if model_type not in best_models_by_type:
                    best_models_by_type[model_type] = result
                elif result['cv_r2'] > best_models_by_type[model_type]['cv_r2']:
                    best_models_by_type[model_type] = result

                if best_overall is None or result['cv_r2'] > best_overall['cv_r2']:
                    best_overall = result
                    best_overall['full_model_name'] = full_model_name

end_time = time.time()
print(f"{end_time - start_time:.2f} секунд")

Перебор комбинаций: 100%|██████████| 31/31 [04:18<00:00,  8.34s/it]

258.65 секунд


In [9]:
results_list = []
for model_name, result in all_results.items():
    results_list.append({
        'model_name': model_name,
        'model_type': result['model_type'],
        'model_specific': result['model_name'],
        'features': str(result['features']),
        'num_features': result['num_features'],
        'cv_r2': result['cv_r2'],
        'cv_std': result['cv_std'],
        'test_r2': result['test_r2'],
        'test_mae': result['test_mae'],
        'combination_id': result['combination_id']
    })

results_df = pd.DataFrame(results_list)
results_df = results_df.sort_values('cv_r2', ascending=False)


print(results_df.to_string(index=False))

                  model_name model_type   model_specific                                                                       features  num_features     cv_r2   cv_std    test_r2  test_mae  combination_id
        GAM_GAM_lam10_comb26        GAM        GAM_lam10         ['time2sleep', 'sleep_duration', 'time_warm_up', 'steps', 'time2wake']             5  0.503300 0.059632   0.628208  3.756171              26
         GAM_GAM_lam10_comb8        GAM        GAM_lam10                                  ['time2sleep', 'sleep_duration', 'time2wake']             3  0.489158 0.119632   0.577187  4.043518               8
         GAM_GAM_lam10_comb0        GAM        GAM_lam10                                               ['time2sleep', 'sleep_duration']             2  0.477449 0.075486   0.391123  4.205014               0
        GAM_GAM_lam10_comb27        GAM        GAM_lam10          ['time2sleep', 'sleep_duration', 'time_warm_up', 'kkal', 'time2wake']             5  0.475956 0.082453   0.618

In [10]:
feature_comb_analysis = []
for comb_id in range(len(feature_combinations)):
    comb_results = results_df[results_df['combination_id'] == comb_id]
    if len(comb_results) > 0:
        feature_comb_analysis.append({
            'combination_id': comb_id,
            'features': feature_combinations[comb_id],
            'num_features': len(feature_combinations[comb_id]),
            'avg_cv_r2': comb_results['cv_r2'].mean(),
            'max_cv_r2': comb_results['cv_r2'].max(),
            'num_models': len(comb_results)
        })

feature_comb_df = pd.DataFrame(feature_comb_analysis)
feature_comb_df = feature_comb_df.sort_values('max_cv_r2', ascending=False)

In [11]:
for i, row in feature_comb_df.head(5).iterrows():
    print(f"{i+1:2d}. Признаки: {row['features']}")
    print(f"Max CV R2: {row['max_cv_r2']:.3f}")

results_df.to_csv('all_models_features_comparison.csv', index=False)
feature_comb_df.to_csv('feature_combinations_analysis.csv', index=False)


if best_overall:
    joblib.dump(best_overall['model'], f"best_overall_model_{best_overall['full_model_name']}.pkl")

for model_type, result in best_models_by_type.items():
    model_filename = f"best_{model_type}_{result['model_name']}_comb{result['combination_id']}.pkl"
    joblib.dump(result['model'], model_filename)

joblib.dump(scaler, 'scaler.pkl')

27. Признаки: ['time2sleep', 'sleep_duration', 'time_warm_up', 'steps', 'time2wake']
Max CV R2: 0.503
 9. Признаки: ['time2sleep', 'sleep_duration', 'time2wake']
Max CV R2: 0.489
 1. Признаки: ['time2sleep', 'sleep_duration']
Max CV R2: 0.477
28. Признаки: ['time2sleep', 'sleep_duration', 'time_warm_up', 'kkal', 'time2wake']
Max CV R2: 0.476
18. Признаки: ['time2sleep', 'sleep_duration', 'time_warm_up', 'time2wake']
Max CV R2: 0.474


['scaler.pkl']

In [13]:
!pip install scikit-optimize -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 2.2 MB/s eta 0:00:00


In [29]:
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args

def time_to_cos(hour, minute=0):
    minutes_total = hour * 60 + minute
    return np.cos(2 * np.pi * minutes_total / 1440)

def cos_to_time(cos_val):
    cos_val = np.clip(cos_val, -1, 1)
    rad = np.arccos(cos_val)
    minutes = rad * 1440 / (2 * np.pi)
    hour = int(minutes // 60)
    minute = int(minutes % 60)
    return hour, minute

best_features = ['time2sleep', 'sleep_duration', 'time_warm_up', 'steps', 'time2wake']
X = df[best_features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler_local = StandardScaler()
X_train_scaled = scaler_local.fit_transform(X_train)

gam_model = LinearGAM(n_splines=10, lam=10)
gam_model.fit(X_train_scaled, y_train)

user_params = {
    'sleep_duration': 450,      # 7.5 часов = 450 минут
    'time_warm_up': 15,
    'steps': 8000,
    'time2wake': 420
}


def predict_for_cos(time_cos, params=user_params):
    features = []
    for feat in best_features:
        if feat == 'time2sleep':
            features.append(time_cos)
        else:
            features.append(params[feat])

    X_input = scaler_local.transform([features])
    return gam_model.predict(X_input)[0]


cos_20 = time_to_cos(20)  # 20:00
cos_2 = time_to_cos(2)    # 02:00


@use_named_args(dimensions=[Real(cos_20, cos_2, name='time_cos')])
def objective(time_cos):
    return -predict_for_cos(time_cos, user_params)


res = gp_minimize(
    func=objective,
    dimensions=[Real(cos_20, cos_2)],
    n_calls=20,
    n_random_starts=5,
    random_state=42,
    verbose=False
)

optimal_cos = res.x[0]
optimal_hour, optimal_minute = cos_to_time(optimal_cos)
optimal_score = -res.fun

current_params = {
    'sleep_duration': X_train['sleep_duration'].mean(),
    'time_warm_up': X_train['time_warm_up'].mean(),
    'steps': X_train['steps'].mean(),
    'time2wake': X_train['time2wake'].mean()
}

current_cos = X_train['time2sleep'].mean()
current_hour, current_min = cos_to_time(current_cos)
current_score = predict_for_cos(current_cos, current_params)


In [30]:
print(current_hour, current_min)

1 34


GAM с регуляризацией lam=10 показала наилучшие результаты среди всех протестированных моделей.

ТОП-3 модели по качеству:
GAM_lam10 — R²=0.503 (CV), R²=0.628 (test), MAE=3.756

GradientBoosting — R²=0.370 (CV), R²=0.363 (test), MAE=4.733

Ridge регрессия — R²=0.239 (CV), R²=0.361 (test), MAE=4.305

Статистическая значимость:
GAM показала наиболее стабильные результаты (стандартное отклонение CV R² = 0.060)

Tree-based модели (RandomForest, GradientBoosting) имели очень высокую дисперсию (std до 0.437), что делает их ненадежными на малых данных

Gaussian Process Regression показал худшие результаты (R²=0.097), видимо переобучилось на малой выборке